In [7]:
# normal imports
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import warnings

# disable warnings
warnings.filterwarnings('ignore')

In [8]:
widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

IntSlider(value=7, continuous_update=False, description='Test:', max=10)

In [2]:
# pybaseball
from pybaseball import statcast_pitcher_pitch_spin

data_spin_all = statcast_pitcher_pitch_spin('2023,2022,2021,2020,2019,2018,2017,2016,2015', minP=100)
print(data_spin_all.columns.tolist())
data_spin_all

['last_name', 'first_name', 'player_id', 'year', 'k_percent', 'bb_percent', 'p_era', 'batting_avg', 'exit_velocity_avg', 'whiff_percent', 'groundballs_percent', 'flyballs_percent', 'popups_percent', 'fastball_avg_spin', 'breaking_avg_spin', 'n_breaking_formatted', 'offspeed_avg_spin', 'n_offspeed_formatted']


,last_name,first_name,player_id,year,k_percent,bb_percent,p_era,batting_avg,exit_velocity_avg,whiff_percent,groundballs_percent,flyballs_percent,popups_percent,fastball_avg_spin,breaking_avg_spin,n_breaking_formatted,offspeed_avg_spin,n_offspeed_formatted
0,Colon,Bartolo,112526,2015,16.7,2.9,4.16,0.281,88.9,14.4,44.1,23.2,5.7,2161,2164.0,10.0,1727.0,7.4
1,Hawkins,LaTroy,115629,2015,21.0,4.3,3.26,0.286,89.7,17.0,55.4,18.2,5.8,2051,2072.0,16.6,1698.0,8.0
2,Wolf,Randy,150116,2015,17.4,9.3,6.23,0.319,89.0,16.2,46.2,17.9,5.1,2032,2176.0,40.2,1669.0,11.1
3,Marquis,Jason,150302,2015,17.1,6.5,6.46,0.330,90.4,21.3,48.8,17.1,3.7,1782,1977.0,19.4,1239.0,21.2
4,Burnett,A.J.,150359,2015,20.5,7.0,3.18,0.275,89.8,21.1,55.0,14.1,4.0,2009,2023.0,29.4,1678.0,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,Woo,Bryan,693433,2023,25.9,7.3,4.75,0.242,87.2,27.8,38.8,28.9,7.9,2179,2358.0,11.8,1713.0,3.6
4310,Elder,Bryce,693821,2023,17.6,7.9,3.46,0.238,89.5,23.7,53.0,20.6,4.8,1998,2398.0,36.7,2032.0,12.1
4311,Pfaadt,Brandon,694297,2023,20.5,6.6,6.91,0.303,90.4,24.6,34.1,30.1,8.5,2445,2668.0,32.9,1946.0,14.7
4312,Shuster,Jared,694363,2023,13.0,11.4,5.00,0.250,89.8,21.3,35.2,29.0,12.4,2136,2204.0,34.2,1492.0,21.7


In [3]:
# massage data - change NaN values to zero, formula to calculate secondary pitch average spin based on pitch usage percentages
data_spin_all = data_spin_all.fillna(0)
data_spin_all["pitch2_avg_spin"] = (data_spin_all['n_breaking_formatted'] / (data_spin_all['n_breaking_formatted'] + data_spin_all['n_offspeed_formatted']) * data_spin_all['breaking_avg_spin']) + \
                                   (data_spin_all['n_offspeed_formatted'] / (data_spin_all['n_breaking_formatted'] + data_spin_all['n_offspeed_formatted']) * data_spin_all['offspeed_avg_spin'])

# change NaN to zero again (formula output to handle pitchers with no secondary pitches)
data_spin_all = data_spin_all.fillna(0)
data_spin_all["pitch2_avg_spin"] = data_spin_all["pitch2_avg_spin"].astype('int')
data_spin_all = data_spin_all.drop(columns=['player_id','breaking_avg_spin','offspeed_avg_spin','n_breaking_formatted','n_offspeed_formatted'])
data_spin_all

,last_name,first_name,year,k_percent,bb_percent,p_era,batting_avg,exit_velocity_avg,whiff_percent,groundballs_percent,flyballs_percent,popups_percent,fastball_avg_spin,pitch2_avg_spin
0,Colon,Bartolo,2015,16.7,2.9,4.16,0.281,88.9,14.4,44.1,23.2,5.7,2161,1978
1,Hawkins,LaTroy,2015,21.0,4.3,3.26,0.286,89.7,17.0,55.4,18.2,5.8,2051,1950
2,Wolf,Randy,2015,17.4,9.3,6.23,0.319,89.0,16.2,46.2,17.9,5.1,2032,2066
3,Marquis,Jason,2015,17.1,6.5,6.46,0.330,90.4,21.3,48.8,17.1,3.7,1782,1591
4,Burnett,A.J.,2015,20.5,7.0,3.18,0.275,89.8,21.1,55.0,14.1,4.0,2009,1943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,Woo,Bryan,2023,25.9,7.3,4.75,0.242,87.2,27.8,38.8,28.9,7.9,2179,2207
4310,Elder,Bryce,2023,17.6,7.9,3.46,0.238,89.5,23.7,53.0,20.6,4.8,1998,2307
4311,Pfaadt,Brandon,2023,20.5,6.6,6.91,0.303,90.4,24.6,34.1,30.1,8.5,2445,2445
4312,Shuster,Jared,2023,13.0,11.4,5.00,0.250,89.8,21.3,35.2,29.0,12.4,2136,1927
